In [ ]:
!git clone https://github.com/Jaish19/Artificial-Intelligence.git

In [36]:
import os
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.metrics import mean_absolute_error

# Disable some pandas warnings
pd.options.mode.chained_assignment = None

# Load dataset
df = pd.read_csv("/content/Artificial-Intelligence/Neural Network Model/house_data.csv")

# Prepare features & target
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

# Scale inputs & outputs
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = Sequential()
model.add(Input(shape=(3,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(
    loss='mean_squared_error',
    optimizer='adam'   # faster & usually better than SGD
)

# Train longer with reasonable batch size
model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=2)

# Ensure save directory exists
save_dir = "/content/Artificial-Intelligence/Neural Network Model"
os.makedirs(save_dir, exist_ok=True)

# Save scalers
joblib.dump(X_scaler, os.path.join(save_dir, "X_scaler.pkl"))
joblib.dump(y_scaler, os.path.join(save_dir, "y_scaler.pkl"))

# Save model in recommended format
model.save(os.path.join(save_dir, "house_value_model.keras"))

# Evaluate
print("Model training results:")
predictions_train = model.predict(X_train, verbose=0)
mse_train = mean_absolute_error(
    y_scaler.inverse_transform(predictions_train),
    y_scaler.inverse_transform(y_train)
)
print(f" - Training Set Error: {mse_train:.2f}")

predictions_test = model.predict(X_test, verbose=0)
mse_test = mean_absolute_error(
    y_scaler.inverse_transform(predictions_test),
    y_scaler.inverse_transform(y_test)
)
print(f" - Test Set Error: {mse_test:.2f}")


Epoch 1/50
235/235 - 2s - 7ms/step - loss: 0.0065
Epoch 2/50
235/235 - 1s - 4ms/step - loss: 3.5026e-04
Epoch 3/50
235/235 - 1s - 3ms/step - loss: 3.5518e-04
Epoch 4/50
235/235 - 1s - 3ms/step - loss: 3.6009e-04
Epoch 5/50
235/235 - 1s - 4ms/step - loss: 3.6011e-04
Epoch 6/50
235/235 - 0s - 2ms/step - loss: 3.6226e-04
Epoch 7/50
235/235 - 1s - 3ms/step - loss: 3.6516e-04
Epoch 8/50
235/235 - 1s - 3ms/step - loss: 3.7649e-04
Epoch 9/50
235/235 - 0s - 2ms/step - loss: 3.8485e-04
Epoch 10/50
235/235 - 1s - 3ms/step - loss: 3.9348e-04
Epoch 11/50
235/235 - 1s - 3ms/step - loss: 3.9538e-04
Epoch 12/50
235/235 - 1s - 3ms/step - loss: 3.8599e-04
Epoch 13/50
235/235 - 1s - 3ms/step - loss: 3.8649e-04
Epoch 14/50
235/235 - 1s - 3ms/step - loss: 3.8625e-04
Epoch 15/50
235/235 - 0s - 2ms/step - loss: 3.8230e-04
Epoch 16/50
235/235 - 0s - 2ms/step - loss: 3.8283e-04
Epoch 17/50
235/235 - 1s - 3ms/step - loss: 3.8630e-04
Epoch 18/50
235/235 - 0s - 2ms/step - loss: 3.7760e-04
Epoch 19/50
235/235 - 0

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
import tensorflow as tf

# =========================
# 1. Set Seeds (Reproducibility)
# =========================
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# =========================
# 2. Load dataset
# =========================
df = pd.read_csv("/content/Artificial-Intelligence/Neural Network Model/house_data.csv")

# Prepare features & target
X = df[["sq_feet", "num_bedrooms", "num_bathrooms"]]
y = df[["sale_price"]]

# =========================
# 3. Scale inputs & outputs
# =========================
X_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))
X[X.columns] = X_scaler.fit_transform(X[X.columns])
y[y.columns] = y_scaler.fit_transform(y[y.columns])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=seed
)

# =========================
# 4. Build Model (with regularization)
# =========================
model = Sequential([
    Input(shape=(3,)),
    Dense(64, activation='relu', kernel_regularizer='l2'),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer='l2'),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer='l2'),
    Dense(1, activation='linear')
])

# Compile with Adam optimizer and custom LR
model.compile(
    loss='mean_squared_error',
    optimizer=Adam(learning_rate=0.001)
)

# =========================
# 5. Add Early Stopping
# =========================
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

# =========================
# 6. Train the Model
# =========================
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    shuffle=True,
    callbacks=[early_stop],
    verbose=2
)

# =========================
# 7. Save Model & Scalers
# =========================
save_dir = "/content/Artificial-Intelligence/Neural Network Model"
os.makedirs(save_dir, exist_ok=True)

joblib.dump(X_scaler, os.path.join(save_dir, "X_scaler.pkl"))
joblib.dump(y_scaler, os.path.join(save_dir, "y_scaler.pkl"))
model.save(os.path.join(save_dir, "house_value_model.keras"))

# =========================
# 8. Evaluate Model
# =========================
print("\nModel training results:")

# Training predictions
predictions_train = model.predict(X_train, verbose=0)
mse_train = mean_absolute_error(
    y_scaler.inverse_transform(y_train),
    y_scaler.inverse_transform(predictions_train)
)
mape_train = mean_absolute_percentage_error(
    y_scaler.inverse_transform(y_train),
    y_scaler.inverse_transform(predictions_train)
)
print(f" - Training Set Error (MAE): {mse_train:.2f}")
print(f" - Training Set Error (MAPE): {mape_train*100:.2f}%")

# Test predictions
predictions_test = model.predict(X_test, verbose=0)
mse_test = mean_absolute_error(
    y_scaler.inverse_transform(y_test),
    y_scaler.inverse_transform(predictions_test)
)
mape_test = mean_absolute_percentage_error(
    y_scaler.inverse_transform(y_test),
    y_scaler.inverse_transform(predictions_test)
)
print(f" - Test Set Error (MAE): {mse_test:.2f}")
print(f" - Test Set Error (MAPE): {mape_test*100:.2f}%")


#Test the trained model

In [39]:
from tensorflow import keras
import joblib
import pandas as pd

# Load trained model (no compile needed for prediction)
model = keras.models.load_model(
    '/content/Artificial-Intelligence/Neural Network Model/house_value_model.h5',
    compile=False
)

# Load scalers
X_scaler = joblib.load('/content/Artificial-Intelligence/Neural Network Model/X_scaler.pkl')
y_scaler = joblib.load('/content/Artificial-Intelligence/Neural Network Model/y_scaler.pkl')

# Define house
house_1 = [800, 2, 2]  # sq_feet, bedrooms, bathrooms
homes = [house_1]

# Convert to DataFrame with column names (to match training)
homes_df = pd.DataFrame(homes, columns=["sq_feet", "num_bedrooms", "num_bathrooms"])

# Scale input
scaled_home_data = X_scaler.transform(homes_df)

# Predict
home_values = model.predict(scaled_home_data, verbose=0)

# Unscale output
unscaled_home_values = y_scaler.inverse_transform(home_values)
predicted_value = unscaled_home_values[0][0]

# Print results
print("House details:")
print(f"- {house_1[0]} sq feet")
print(f"- {house_1[1]} bedrooms")
print(f"- {house_1[2]} bathrooms")
print(f"Estimated value: ${predicted_value:,.2f}")


House details:
- 800 sq feet
- 2 bedrooms
- 2 bathrooms
Estimated value: $183,724.83
